# Install, Import, and Log In

In [228]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.datasets import fashion_mnist

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

### Step 1: Import W&B and Login

In [229]:
import wandb
from wandb.keras import WandbCallback

project_name = 'project-fashion-mnist'
# %env WANDB_API_KEY='1af04e33e3d441eb82eb612e2c001eddec29bccb'

# wandb.login(relogin=True)

> Side note: If this is your first time using W&B or you are not logged in, the link that appears after running `wandb.login` will take you to sign-up/login page. Signing up is easy!

# Download and Prepare the Dataset


In [230]:
from class_names import class_names

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train / 255., x_test / 255.

input_shape = (28, 28, 1)
num_classes = len(class_names)

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)

Shape of x_train:  (60000, 28, 28)
Shape of y_train:  (60000,)
Shape of x_test:  (10000, 28, 28)
Shape of y_test:  (10000,)


# Define the Model

Here, we define a standard CNN (with convolution and max-pooling) in Keras.

In [231]:
def Model():
    inputs = keras.layers.Input(shape=input_shape)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(inputs)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)

    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dense(28, activation='relu')(x)

    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    return keras.models.Model(inputs=inputs, outputs=outputs, )

# Train the Model

In [232]:
# Initialize wandb with your project name
run = wandb.init(project=project_name,
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "fashion_mnist"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

wandb: Currently logged in as: eshcharc. Use `wandb login --relogin` to force relogin


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 28)        280       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 28)        7084      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 28)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 28)        7084      
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 28)          7084      
                                                             

In [ ]:
# We focus on a subset of images, since this is for human review
val_images, val_labels = x_test[:32], y_test[:32]

_ = model.fit(x_train, y_train,
              epochs=config.epochs, 
              batch_size=config.batch_size,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type='image', 
                                       validation_data=(val_images, val_labels), 
                                       labels=class_names)])

run.finish()

In [272]:
import keras_tuner as kt
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.losses import SparseCategoricalCrossentropy

def model_builder(hp):
  hp_conv_units = hp.Int('hp_conv_units', min_value=32, max_value=64, step=4)
  hp_kernel = hp.Int('hp_kernel', min_value=3, max_value=6, step=1)
  hp_dropout_rate = hp.Choice('hp_dropout_rate', [0.25, 0.5])
  hp_pool_size = hp.Int('hp_pool_size', min_value=3, max_value=5)
  hp_dense_units = hp.Int('hp_dense_units', min_value=100, max_value=200, step=10)
  hp_learning_rate = hp.Float('learning_rate', min_value=0.001, max_value=0.01, sampling="log")
  ht_activation = hp.Choice("activation", ["relu", "tanh"])

  model = keras.Sequential([
    Input(shape=input_shape),

    Conv2D(filters=hp_conv_units, kernel_size=hp_kernel, activation=ht_activation, padding="SAME"),
    Conv2D(filters=hp_conv_units, kernel_size=hp_kernel, activation=ht_activation, padding="SAME"),
    MaxPooling2D(pool_size=hp_pool_size, padding="SAME"),
    # Dropout(rate=hp_dropout_rate),

    Conv2D(filters=hp_conv_units*2, kernel_size=hp_kernel*2, activation=ht_activation, padding="SAME"),
    Conv2D(filters=hp_conv_units*2, kernel_size=hp_kernel*2, activation=ht_activation, padding="SAME"),
    MaxPooling2D(pool_size=hp_pool_size, padding="SAME"),
    # Dropout(rate=hp_dropout_rate),

    Conv2D(filters=hp_conv_units*4, kernel_size=hp_kernel*4, activation=ht_activation, padding="SAME"),
    Conv2D(filters=hp_conv_units*4, kernel_size=hp_kernel*4, activation=ht_activation, padding="SAME"),
    MaxPooling2D(pool_size=hp_pool_size, padding="SAME"),
    # Dropout(rate=hp_dropout_rate),

    Flatten(),
    Dense(units=hp_dense_units, activation='relu'),
    Dense(units=num_classes, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(
                      from_logits=False,
                      ignore_class=None,
                      name='sparse_categorical_crossentropy'
                ),
                metrics=['accuracy'])
  
  return model

model = model_builder(kt.HyperParameters())
# model.summary()

In [273]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

tuner = kt.Hyperband(model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='hp_search',
    project_name='predict_fashion_item-6')

es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
mc = ModelCheckpoint('./fashion_mnist.hd5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

tuner.search(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[es, mc])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

INFO:tensorflow:Reloading Oracle from existing project hp_search\predict_fashion_item-6\oracle.json


INFO:tensorflow:Reloading Oracle from existing project hp_search\predict_fashion_item-6\oracle.json



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
48                |?                 |hp_conv_units
4                 |?                 |hp_kernel
0.5               |?                 |hp_dropout_rate
4                 |?                 |hp_pool_size
160               |?                 |hp_dense_units
0.0035543         |?                 |learning_rate
relu              |?                 |activation
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2
  26/1500 [..............................] - ETA: 17:56 - loss: 2.2765 - accuracy: 0.1058

KeyboardInterrupt: 

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

# Save Trained Best Model

In [ ]:
model.save('./fashion_mnist.hd5')